In [1]:
import simons_data_science_library as sdl
import pandas as pd
from simons_data_science_library import db_utils as rdc

In [4]:
table = rdc.RDSDatabaseConnector.csv_load('gold_weightings.txt', ' ')

our df
   Guest  Ligand_clash  Ligand_torsion  Part_buried  Non-polar  \
0      1           0.0             0.0      -2.1090   -43.3137   
1      2           0.0             0.0      -3.0465   -27.4388   
2      3           0.0             0.0      -2.4099   -41.2634   
3      4           0.0             0.0      -2.2600   -32.0514   
4      5           0.0             0.0      -2.7351   -40.6438   

   Ligand_flexibility  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  
